### Coiled for ETL #1
# Process 75GB of JSON Data into Parquet in under 10 minutes

The *Coiled for ETL* series demonstrates how Coiled can speed up your existing ETL workflows for larger-than-memory datasets.

This edition presents a common ETL use case in which you will: 
- **Extract** raw .json data scraped from the web, 
- **Transform** it into tabular DataFrame format optimal for EDA and/or ML,
- **Load** it to a cloud object storage as parquet for future reference.

> If you're not familiar with Parquet and its benefits over other formats like .json and .csv, check out [this blog](https://coiled.io/blog/parquet-column-pruning-predicate-pushdown/) 

We’ll be working with data from the [Github Archive project](https://www.gharchive.org/) for the year 2015. This dataset logs all public activity on Github and takes up ~75GB in uncompressed form. That means you most likely can’t process it entirely on your local machine. 

Upon completing this notebook, you will be able to:
1. Build and test your ETL workflow locally first, using a single test file representing 1 hour of GH activity data
2. Scale that same workflow out to the cloud using Dask and Coiled to process the entire dataset

*Spoiler alert*: you’ll be running the exact same code in both cases, just changing the place where the computations are run.

### TO DO
1. Mention: Use environment.yml in repo locally
2. Remove path to public S3 bucket in final version? Check with Samantha.

## Building your ETL Pipeline Locally

Before applying any kind of transformation to a dataset, you'll likely first want to get a sense for what it contains and the kinds of data wrangling you might have to do. That's why we'll build our ETL pipeline locally first by:

1. Extracting the data
2. Transforming it into a DataFrame
3. Loading it to a local directory in Parquet file format

### 1. Extract the Data
We'll start by inspecting a single file from the Github Archive. This represents 1 hour of data and takes up ~5MB of data. There's no need to work with any kind of parallel or cloud computing here, so you can iterate locally for now.

> *Only scale out to the cloud if and when necessary to avoid unnecessary costs and code complexity.*

The Github Archive data can be accessed via URLs.  Let’s **wget** a single file.

In [1]:
!wget https://data.gharchive.org/2015-01-01-15.json.gz

--2021-09-07 10:38:55--  https://data.gharchive.org/2015-01-01-15.json.gz
Resolving data.gharchive.org (data.gharchive.org)... 172.67.168.206, 104.21.46.175
Connecting to data.gharchive.org (data.gharchive.org)|172.67.168.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3844072 (3.7M) [application/gzip]
Saving to: '2015-01-01-15.json.gz.6'

2015-01-01-15.json. 100%[===================>]   3.67M  15.0MB/s    in 0.2s    

2021-09-07 10:38:55 (15.0 MB/s) - '2015-01-01-15.json.gz.6' saved [3844072/3844072]



The data is stored in gzipped .json format. Let's start by loading it into a Dask Bag.

In [1]:
import dask.bag as db
import ujson

In [3]:
# load test data into dask bag and inspect first entry
records = db.read_text("2015-01-01-15.json.gz").map(ujson.loads)
records.take(1)

({'id': '2489651045',
  'type': 'CreateEvent',
  'actor': {'id': 665991,
   'login': 'petroav',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/petroav',
   'avatar_url': 'https://avatars.githubusercontent.com/u/665991?'},
  'repo': {'id': 28688495,
   'name': 'petroav/6.828',
   'url': 'https://api.github.com/repos/petroav/6.828'},
  'payload': {'ref': 'master',
   'ref_type': 'branch',
   'master_branch': 'master',
   'description': "Solution to homework and assignments from MIT's 6.828 (Operating Systems Engineering). Done in my spare time.",
   'pusher_type': 'user'},
  'public': True,
  'created_at': '2015-01-01T15:00:00Z'},)

### 2. Transform into DataFrame


#### Subsetting the data

There are several different schemas overlapping here, which means we can't simply cast this into a pandas or Dask DataFrame. We **can** filter out one subset, though, and work with that. 

Let's take a look at the frequencies of the different **types** of records in this test sample.

In [4]:
records.pluck("type").frequencies().compute()

[('CreateEvent', 1471),
 ('PushEvent', 5815),
 ('WatchEvent', 1230),
 ('ReleaseEvent', 60),
 ('PullRequestEvent', 474),
 ('IssuesEvent', 545),
 ('ForkEvent', 355),
 ('GollumEvent', 61),
 ('IssueCommentEvent', 844),
 ('DeleteEvent', 260),
 ('PullRequestReviewCommentEvent', 136),
 ('CommitCommentEvent', 73),
 ('MemberEvent', 25),
 ('PublicEvent', 2)]

PushEvents seem popular, and also include interesting information. Let's start there.

In [5]:
records.filter(lambda record: record["type"] == "PushEvent").take(1)

({'id': '2489651051',
  'type': 'PushEvent',
  'actor': {'id': 3854017,
   'login': 'rspt',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/rspt',
   'avatar_url': 'https://avatars.githubusercontent.com/u/3854017?'},
  'repo': {'id': 28671719,
   'name': 'rspt/rspt-theme',
   'url': 'https://api.github.com/repos/rspt/rspt-theme'},
  'payload': {'push_id': 536863970,
   'size': 1,
   'distinct_size': 1,
   'ref': 'refs/heads/master',
   'head': '6b089eb4a43f728f0a594388092f480f2ecacfcd',
   'before': '437c03652caa0bc4a7554b18d5c0a394c2f3d326',
   'commits': [{'sha': '6b089eb4a43f728f0a594388092f480f2ecacfcd',
     'author': {'email': '5c682c2d1ec4073e277f9ba9f4bdf07e5794dabe@rspt.ch',
      'name': 'rspt'},
     'message': 'Fix main header height on mobile',
     'distinct': True,
     'url': 'https://api.github.com/repos/rspt/rspt-theme/commits/6b089eb4a43f728f0a594388092f480f2ecacfcd'}]},
  'public': True,
  'created_at': '2015-01-01T15:00:01Z'},)

#### Flattening the data

Recall that we want to get this data into a format that makes sense for EDA (Exploratory Data Analysis) and ML, which means we'll need to get it into a pandas or Dask DataFrame. To do that, we'll have to flatten down this data so that pandas operations can be applied to it. While we're at it, let's also filter out some of the attributes so that we're only working with the data we really care about.

The function below extracts information about the commits in each PushEvent. This flattens out the nested data into a tabular format where each row represents a single commit.

In [6]:
# define function that extracts relevant data
def process(record):
    try:
        for commit in record["payload"]["commits"]:
            yield {
                "user": record["actor"]["login"],
                "repo": record["repo"]["name"],
                "created_at": record["created_at"],
                "message": commit["message"],
                "author": commit["author"]["name"],
            }
    except KeyError:
        pass

Let's test that function on a single record to confirm it's working as expected.

In [7]:
# apply processing function to 1 record
[record] = records.filter(lambda record: record["type"] == "PushEvent").take(1)
list(process(record))

[{'user': 'rspt',
  'repo': 'rspt/rspt-theme',
  'created_at': '2015-01-01T15:00:01Z',
  'message': 'Fix main header height on mobile',
  'author': 'rspt'}]

Now let's apply this to all the PushEvent records.

In [26]:
# apply processing function to all records in test file
flattened = records.filter(lambda record: record["type"] == "PushEvent").map(process).flatten()
flattened.take(3)

({'user': 'soumith',
  'repo': 'soumith/fbcunn',
  'created_at': '2015-01-01T01:00:00Z',
  'message': 'back to old structure, except lua files moved out',
  'author': 'Soumith Chintala'},
 {'user': 'soumith',
  'repo': 'soumith/fbcunn',
  'created_at': '2015-01-01T01:00:00Z',
  'message': '...',
  'author': 'Soumith Chintala'},
 {'user': 'soumith',
  'repo': 'soumith/fbcunn',
  'created_at': '2015-01-01T01:00:00Z',
  'message': '...',
  'author': 'Soumith Chintala'})

#### Convert to Dataframe
We're now ready to convert this flattened data into a DataFrame. We'll convert straight into a Dask DataFrame since we're already working with a Dask Bag and will be applying more Dask operations later in this notebook. Depending on your workflow, you could also work with pandas DataFrames here.

In [27]:
# cast flattened json data into dataframe
df = flattened.to_dataframe()
df.head()

,user,repo,created_at,message,author
0,soumith,soumith/fbcunn,2015-01-01T01:00:00Z,"back to old structure, except lua files moved out",Soumith Chintala
1,soumith,soumith/fbcunn,2015-01-01T01:00:00Z,...,Soumith Chintala
2,soumith,soumith/fbcunn,2015-01-01T01:00:00Z,...,Soumith Chintala
3,soumith,soumith/fbcunn,2015-01-01T01:00:00Z,...,Soumith Chintala
4,radix,radix/effect,2015-01-01T01:00:00Z,put the auto-generated API docs in the reposit...,Christopher Armstrong


In [10]:
# check number of rows
len(df)

10109

Excellent, we've managed to convert the .json test file into a Dask DataFrame. 

Note that the DataFrame contains just over 10k entries. This is because we've flattened the nested PushEvents into single commits and apparently some of the 5815 PushEvents in **records** contained multiple commits each.

### 3. Load to Disk as Parquet

We're now ready to write our DataFrame into a .parquet file.

In [11]:
# write test dataframe to local directory as parquet
df.to_parquet(
    "test.parq", 
    engine="pyarrow",
    compression="snappy"
)

As a final sanity-check, let's import the saved parquet file as a dataframe.

In [12]:
import dask.dataframe as dd

df = dd.read_parquet(
    "test.parq",
    engine="pyarrow",
)

df.head()

,user,repo,created_at,message,author
0,rspt,rspt/rspt-theme,2015-01-01T15:00:01Z,Fix main header height on mobile,rspt
1,izuzero,izuzero/xe-module-ajaxboard,2015-01-01T15:00:01Z,#20 게시글 및 댓글 삭제 시 새로고침이 되는 문제 해결\n\n원래 의도는 새로고...,Eunsoo Lee
2,winterbe,winterbe/streamjs,2015-01-01T15:00:03Z,"Add comparator support for min, max operations",Benjamin Winterberg
3,hermanwahyudi,hermanwahyudi/selenium,2015-01-01T15:00:03Z,Update README.md,Herman
4,jdilt,jdilt/jdilt.github.io,2015-01-01T15:00:03Z,refine index page and about page,jdilt


Great, that's looking good!

## Scaling to the Cloud

Now that we have figured out our flow locally, let's build a workflow that will collect the data for a full year (~75GB uncompressed), process it, and save it to cloud object storage.

In this section you will:
1. Create a list of filenames to extract
2. Spin up a Coiled cluster
3. Execute the ETL pipeline on the entire dataset

### 1. Get a list of dates and filenames

In the first section, we downloaded a testing file from the internet to our local computer and then wrapped a Dask Bag around it. Now we're going to need to create a Dask Bag around a list of files on the internet. 

To do that, let's create a list of filenames of all dates in the last year. 

In [2]:
# Get a list of dates of the last year, turn this list into a list of filenames like what is above
import datetime

start = datetime.datetime.strptime("01-01-2015", "%d-%m-%Y")
end = datetime.datetime.strptime("31-12-2015", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
prefix = "https://data.gharchive.org/"
filenames = []
for date in date_generated:
    for hour in range(1,24):
        filenames.append(prefix + date.strftime("%Y-%m-%d") + '-' + str(hour) + '.json.gz')

# Check filenames look ok
filenames[:100:10]

['https://data.gharchive.org/2015-01-01-1.json.gz',
 'https://data.gharchive.org/2015-01-01-11.json.gz',
 'https://data.gharchive.org/2015-01-01-21.json.gz',
 'https://data.gharchive.org/2015-01-02-8.json.gz',
 'https://data.gharchive.org/2015-01-02-18.json.gz',
 'https://data.gharchive.org/2015-01-03-5.json.gz',
 'https://data.gharchive.org/2015-01-03-15.json.gz',
 'https://data.gharchive.org/2015-01-04-2.json.gz',
 'https://data.gharchive.org/2015-01-04-12.json.gz',
 'https://data.gharchive.org/2015-01-04-22.json.gz']

In [14]:
# # Can also create filenames using nested comprehensions
# import itertools

# filenames_2 = [[prefix + date.strftime("%Y-%m-%d") + '-' + str(hour) + '.json.gz' for hour in range(1,24)] for date in date_generated]
# filenames_flat = list(itertools.chain(*filenames_2))

### 2. Create Dask Cluster on the Cloud

Now we launch a Dask cluster in the cloud that can run our ETL pipeline on the entire dataset, i.e. 1 year of Github archive data.  

You'll need to make a software environment with the correct libraries so that the workers in your cluster are able to execute our computations. Let's do that first.

> *You will need a Free Coiled account for this section. Follow the [Getting Started](https://docs.coiled.io/user_guide/getting_started.html) guide in our docs to create one using just your Github credentials.*

In [21]:
import coiled

In [33]:
%%capture
# create Coiled software environment
coiled.create_software_environment(
    name="github-parquet",
    conda=["dask", "pyarrow", "s3fs", "ujson", "requests", "lz4", "fastparquet"],
)

You can also create Coiled software environments using Docker images, environment.yml (conda) or requirements.txt (pip) files. For more information, check out the [Coiled Docs](https://docs.coiled.io/user_guide/software_environment_creation.html).

In [22]:
# spin up a Coiled cluster
cluster = coiled.Cluster(
    name="github-parquet", #name your cluster for future reference
    software="coiled-examples/github-parquet", #specify the software environment
    n_workers=10, #let's start with 10 workers
    shutdown_on_close=False, #this keeps the cluster running if your Python session closes
)

Output()

Your 'blog-notebooks' and 'coiled-examples' accounts are using the ECS backend.
After September 16th, accounts using the ECS backend will be migrated to the default AWS VM backend. For more information, refer to the backend documentation and the FAQ:

https://docs.coiled.io/user_guide/backends
https://docs.coiled.io/user_guide/faq.html#backends
Found software environment build
Created FW rules: coiled-dask-rrpelgr71-42865-firewall
Created scheduler VM: coiled-dask-rrpelgr71-42865-scheduler (type: t3a.medium, ip: ['35.153.126.12'])


In [23]:
# connect Dask to your Coiled cluster
from dask.distributed import Client
client = Client(cluster)
client

/Users/rpelgrim/mambaforge/envs/etl/lib/python3.9/site-packages/distributed/client.py:1100: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+----------------+----------------+
| Package     | client        | scheduler      | workers        |
+-------------+---------------+----------------+----------------+
| dask        | 2021.09.0     | 2021.08.0      | 2021.08.0      |
| distributed | 2021.09.0     | 2021.08.0      | 2021.08.0      |
| numpy       | 1.21.1        | 1.20.3         | 1.20.3         |
| python      | 3.9.6.final.0 | 3.8.11.final.0 | 3.8.11.final.0 |
+-------------+---------------+----------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tls://10.4.1.210:8786' processes=3 threads=6, memory=22.69 GiB>

### 3. Transform and Load data to Cloud Object Storage

The moment we've all been waiting for! Your cluster is up, so you're all set to run the ETL pipeline we built above on the entire dataset. Note that this requires just 2 subtle changes:
1. pass the entire list **filenames** to `db.read_text` 
2. point `df.to_parquet` to write to your s3 bucket

In [ ]:
%%time
# read in json data
records = db.read_text(filenames).map(ujson.loads)

# filter out only PushEvents
push = records.filter(lambda record: record["type"] == "PushEvent")

# process into single commit entries
processed = push.map(process)

# flatten and cast to dataframe
df = processed.flatten().to_dataframe()

# write to parquet
df.to_parquet(
    's3://coiled-datasets/etl/test.parq',
    engine='pyarrow',
    compression='snappy'
)

Excellent, that works. But it took quite some time. 

Let's scale our cluster up to see if we can get better performance. We'll use the `cluster.scale()` command as well as a call to `client.wait_for_workers` which will block activity until all of the workers are online. This way we can be sure that we're throwing all the muscle we have at our computation.

In [46]:
# double n_workers
cluster.scale(20)

# this blocks activity until the specified number of workers have joined the cluster
client.wait_for_workers(20)

In [ ]:
%%time
# re-run etl pipeline
records = db.read_text(filenames).map(ujson.loads)
push = records.filter(lambda record: record["type"] == "PushEvent")
processed = push.map(process)
df = processed.flatten().to_dataframe()
df.to_parquet(
    's3://coiled-datasets/etl/test.parq',
    engine='pyarrow',
    compression='snappy'
)

Less than 10 minutes to process 75GB of data, great job!

# Let's Recap

In this notebook we performed a common ETL workflow: converting raw JSON data into a flattened DataFrame and storing in the efficient Parquet file format on a cloud object store. 

We performed this first on a single test file locally and then scaled this out to run on the cloud using Dask clusters on Coiled in order to process the entire 75GB dataset for the year 2015.

Main takeaways:
- Coiled allows you to scale common ETL workflows to larger-than-memory datasets.
- Only scale to the cloud if and when you need to. Cloud computing comes with its own set of challenges and overhead. So be strategic about deciding if and when to import Coiled and spin up a cluster.
- Scale up your cluster for increased performance. We cut the runtime of the ETL function in half by scaling our cluster from 10 to 20 workers.

If you have any questions or suggestions for future material, feel free to drop us a line at support@coiled.io or in our [Coiled Community Slack channel](https://join.slack.com/t/coiled-users/shared_invite/zt-hx1fnr7k-In~Q8ui3XkQfvQon0yN5WQ). We'd love to hear from you!